In [5]:
%reload_ext autoreload
%autoreload 2

In [6]:
from classes import *
import itertools
import pandas as pd
from tqdm import tqdm

### Scratch Work

In [ ]:
spendstrat = SpendingStrategy(
    base_spending=0.5,
    retirement_saving=0.15,
    disp_spend=0.15,
    disp_give=0.1
)

In [ ]:
genstrat = GenerosityStrategy(
    straight_percent=0.5,
    investment_draw_down_rate=0.04,
    legacy_give_percent=0.2
)

In [ ]:
salary = Salary(112000)
pm = PortfolioManager(
    person_age=28,
    spendstrat=spendstrat,
    genstrat=genstrat,
    salary=salary,
)

pm.init_retirement_savings(9450+2241+27705) # 12/23/23
pm.init_giving_savings(4150)# 12/23/23
pm.df

In [ ]:
for i in range(12 * (65-28)):
    years_from_start = (i+1) / 12
    pm.simulate_month(years_from_start)
print(pm.df["Total Giving"].sum())
display(pm.df)

### Optimizer

In [20]:
spendstrat_params = {
    "base_spending": [0.5],
    "retirement_saving": [0.15],
    "disp_spend": [0.15],
    "disp_give": [0.1, 0.3, 0.4, 0.5]
}

genstrat_params = {
    "straight_percent": [0.1, 0.3, 0.5, 0.7, 0.9],
    "investment_draw_down_rate": [0.01, 0.015, 0.02, 0.025, 0.03],
    "legacy_give_percent": [0.2]
}

analysis_df = pd.DataFrame(columns=[
    "base_spending",
    "retirement_saving",
    "disp_spend",
    "disp_give",
    "straight_percent",
    "investment_draw_down_rate",
    "legacy_give_percent",
    "total_giving"
])

In [21]:
# Generate all combinations of parameters
spendstrat_combinations = list(itertools.product(
    spendstrat_params["base_spending"],
    spendstrat_params["retirement_saving"],
    spendstrat_params["disp_spend"],
    spendstrat_params["disp_give"],
))

genstrat_combinations = list(itertools.product(
    genstrat_params["straight_percent"],
    genstrat_params["investment_draw_down_rate"],
    genstrat_params["legacy_give_percent"],
))

for spendstrat_tuple in spendstrat_combinations:
    for genstrat_tuple in tqdm(genstrat_combinations, "genstrats analyzed"):
        bs, rs, ds, dg = spendstrat_tuple
        sp, iddr, lgp = genstrat_tuple
        salary = Salary(112000)
        spendstrat = SpendingStrategy(
            base_spending=bs,
            retirement_saving=rs,
            disp_spend=ds,
            disp_give=dg
        )
        genstrat = GenerosityStrategy(
            straight_percent=sp,
            investment_draw_down_rate=iddr,
            legacy_give_percent=lgp
        )
        pm = PortfolioManager(
            person_age=28,
            spendstrat=spendstrat,
            genstrat=genstrat,
            salary=salary,
        )
        pm.init_retirement_savings(9450+2241+27705) # 12/23/23
        pm.init_giving_savings(4150)# 12/23/23

        for i in range(12 * (80-28)):
            years_from_start = (i+1) / 12
            pm.simulate_month(years_from_start)

        total_giving = (
            pm.df["Total Giving"].sum() 
            + pm.ia.reverse_adjust(pm.giving_investment.total, years_from_start)
            + pm.ia.reverse_adjust(pm.retirement_investment.total * pm.genstrat.legacy_give_percent, years_from_start)
            )
        new_row = {
            "base_spending": bs,
            "retirement_saving": rs,
            "disp_spend": ds,
            "disp_give": dg,
            "straight_percent": sp,
            "investment_draw_down_rate": iddr,
            "legacy_give_percent": lgp,
            "total_giving": total_giving
        }
        if len(analysis_df) == 0:
            analysis_df = pd.DataFrame([new_row])
        else:
            analysis_df = pd.concat([analysis_df, pd.DataFrame([new_row])], ignore_index=True)

spendstrats analyzed: 100%|██████████| 9/9 [07:20<00:00, 49.00s/it]


In [22]:
eval_df = analysis_df.copy()
eval_df = eval_df.sort_values(by="total_giving", ascending=False)
eval_df = eval_df.round(2)
display(eval_df.head(7))

,base_spending,retirement_saving,disp_spend,disp_give,straight_percent,investment_draw_down_rate,legacy_give_percent,total_giving
40,0.5,0.10,0.1,0.5,0.1,0.01,0.2,2714707.47
100,0.5,0.15,0.1,0.5,0.1,0.01,0.2,2475232.66
44,0.5,0.10,0.1,0.5,0.3,0.01,0.2,2432881.76
41,0.5,0.10,0.1,0.5,0.1,0.03,0.2,2379311.05
20,0.5,0.10,0.1,0.4,0.1,0.01,0.2,2294876.73
104,0.5,0.15,0.1,0.5,0.3,0.01,0.2,2234369.28
101,0.5,0.15,0.1,0.5,0.1,0.03,0.2,2184304.42
